
# 自动捡货模拟(Simulation)

### 程式修改自Roberto Sannazzaro, [‘How to Automatize a Warehouse Robot’](https://medium.datadriveninvestor.com/get-started-with-q-learning-with-python-how-to-automatize-a-warehouse-robot-7bfae0180301)

## Introduction 

The use of robotics is constantly expanding in every business sector, automation takes repetitive tasks and aims to automatize them, in order to optimize processes and cut costs.
In 2012 Amazon purchased Kiva Systems, a company which developed warehouse robots and related technologies, and which was acquired for $775 million. Moreover many other companies implement robots in their warehouses, even robots that can work in 3 dimensions.

<br/><br/>

![Autonomous warehouse robot](https://media.giphy.com/media/s0urqX40zokIo/giphy.gif)




## 仓库布置图：
<img src="https://i.ibb.co/LrTfrgc/warehouse.png">

## 载入套件

In [1]:
import numpy as np

## 定义环境(environment)

In [2]:
# 位置编码
location_to_state = {'A': 0,
                     'B': 1,
                     'C': 2,
                     'D': 3,
                     'E': 4,
                     'F': 5,
                     'G': 6,
                     'H': 7,
                     'I': 8,
                     'J': 9,
                     'K': 10,
                     'L': 11}

## 行动空间

In [3]:
actions = [0,1,2,3,4,5,6,7,8,9,10,11]

## 定义行动限制，假设G点有最高优先度, 故奖励设为1000

In [4]:
# 行动限制，1: 可到达，0:不可到达
R = np.array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0, 0, 1000, 1, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]])

## 依TD(1) 演算法更新行动值函数

In [5]:
# 参数设定
gamma = 0.75
alpha = 0.9

# 行动值函数初始值为 0
Q = np.array(np.zeros([12,12]))

# 训练 1000 周期
for i in range(1000):
    # 随机起始点
    current_state = np.random.randint(0,12)
    playable_actions = []
    for j in range(12):
        if R[current_state, j] > 0:
            playable_actions.append(j)
    # 任意行动
    next_state = np.random.choice(playable_actions)
    # 更新行动值函数
    TD = R[current_state, next_state] + gamma*Q[next_state, \
            np.argmax(Q[next_state,])] - Q[current_state, next_state]
    Q[current_state, next_state] = Q[current_state, next_state] + alpha*TD

## 显示更新结果：越靠近G点，值函数越高

In [6]:
import pandas as pd

q_values = pd.DataFrame(Q, columns=[location for location in location_to_state])
s = q_values.round().style.background_gradient(cmap='GnBu')
s

,A,B,C,D,E,F,G,H,I,J,K,L
0,0.000000,1688.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1267.000000,0.000000,2250.000000,0.000000,0.000000,1267.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,1688.000000,0.000000,0.000000,0.000000,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2251.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,715.000000,0.000000,0.000000,0.000000
5,0.000000,1688.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,951.000000,0.000000,0.000000
6,0.000000,0.000000,2251.000000,0.000000,0.000000,0.000000,3999.000000,2249.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,1688.000000,0.000000,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,1689.000000
8,0.000000,0.000000,0.000000,0.000000,537.000000,0.000000,0.000000,0.000000,0.000000,951.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,1267.000000,0.000000,0.000000,715.000000,0.000000,1267.000000,0.000000


## 重新定义行动限制

In [7]:
R = np.array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]])

## 定义代码与位置对照表

In [8]:
state_to_location = {state: location for location, 
                     state in location_to_state.items()}
state_to_location            

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L'}

## 定义路由训练函数

In [9]:
def route(starting_location, ending_location):
    # starting_location, ending_location：起点、终点
    # 位置转换为代码
    ending_state = location_to_state[ending_location]
    # 终点有最高优先度
    R_new = np.copy(R)
    R_new[ending_state, ending_state] = 1000
    
    # 策略评估：训练 1000 周期
    Q = np.array(np.zeros([12,12]))
    for i in range(1000):
        current_state = np.random.randint(0,12)
        playable_actions = []
        for j in range(12):
            if R_new[current_state, j] > 0:
                playable_actions.append(j)
        # 任意行动
        next_state = np.random.choice(playable_actions)
        # 更新行动值函数
        TD = R_new[current_state, next_state] + gamma * \
            Q[next_state, np.argmax(Q[next_state,])] - Q[current_state, next_state]
        Q[current_state, next_state] = Q[current_state, next_state] + alpha * TD
        
    # 策略改善：依TD找寻最佳路由
    route = [starting_location]
    next_location = starting_location
    while (next_location != ending_location):
        starting_state = location_to_state[starting_location]
        next_state = np.argmax(Q[starting_state,])
        next_location = state_to_location[next_state]
        route.append(next_location)
        starting_location = next_location
    return route

## 测试 E --> G 最佳路由

In [10]:
route('E', 'G')

['E', 'I', 'J', 'K', 'L', 'H', 'G']

<img src="https://i.ibb.co/VJ1KKcR/warehouse-1.png">

In [11]:
# 测试 A --> K 最佳路由
route('A', 'K')

['A', 'B', 'F', 'J', 'K']

In [12]:
# 3 个点的路由
def best_route(starting_location, intermediary_location, \
               ending_location):
    # 3 个点的路由 = 2 个点的路由 + 2 个点的路由
    return route(starting_location, intermediary_location) + \
            route(intermediary_location, ending_location)[1:]

## 测试 E --> K --> G 最佳路由

In [13]:
best_route('E', 'K', 'G')

['E', 'I', 'J', 'K', 'L', 'H', 'G']

<img src="https://i.ibb.co/k2HBnyy/warehouse-2.png">

In [14]:
# 测试 A --> G --> K 最佳路由
initial = "A" 
intermediary = "G" 
final = "K" 
best = best_route(initial, intermediary, final)
print('最佳路由: ')
print(*best, sep=', ')

最佳路由: 
A, B, C, G, H, L, K


## References 

* [An introduction to Q-Learning](https://www.freecodecamp.org/news/an-introduction-to-q-learning-reinforcement-learning-14ac0b4493cc/)
* [Reinforcement learning](https://medium.com/machine-learning-for-humans/reinforcement-learning-6eacf258b265)
*[Math of Q-Learning](https://medium.com/datadriveninvestor/math-of-q-learning-python-code-5dcbdc49b6f6)